In [1]:
from pypdf import PdfReader
import pandas as pd

In [2]:
# read the file you're working with
reader = PdfReader('Képviselők_20250228.pdf')

In [3]:
print(len(reader.pages))

2258


In [4]:
# to inspect one page's text

page = reader.pages[53]
text = page.extract_text()
print(text)
t_lines = list(text.split('\n'))

II. Nagy értékű3 ingóságok
1. Gépjárművek:
a) személygépkocsi: Honda Accord Type-S 2.4 i típus
    a szerzés ideje, jogcíme: 2014. 04. 28.,  adásvétel
b) tehergépjármű, autóbusz: "-" típus
    a szerzés ideje, jogcíme: -----
c) motorkerékpár: "-" típus
    a szerzés ideje, jogcíme: -----
2. Vízi vagy légi jármű:
jellege: "-"
    típusa: -----
    a szerzés ideje, jogcíme: -----
3. Védett műalkotás, védett gyűjtemény:
a) egyedi alkotások:
"-" megnevezés ----- db
a szerzés ideje, jogcíme: -----
b) gyűjtemény:
"-" megnevezés ----- db
a szerzés ideje, jogcíme: -----
4. Egyéb ingóság:
megnevezés: "-"
    a szerzés ideje, jogcíme: -----
5. Értékpapírban elhelyezett megtakarítás vagy egyéb befektetés (részvény, kötvény,
részjegy, kincstárjegy, vagyonjegy, részesedés magántőkealapban, biztosítás stb.):
megnevezés: "-"
névérték, biztosítási összeg: -----
6. Takarékbetétben elhelyezett megtakarítás4: 12.164.448.-Ft Ft
3 Nagy értékűnek minősül az ingóság, ha értéke eléri az 5 millió forintot.
4 K

In [20]:
# inspect one line


'típus' in t_lines[5].strip()


False

In [7]:
#INGATLAN RAW DATA

# Define a dataframe to store your structured data

vny_2025 = pd.DataFrame(columns = ['oldal', 'sor', 'PID', 'name', 'ing_sz', 'ingatlan', 'terület', 'művelési ág', 'rendeltetés, alapterület', 'jogi jelleg', 
                                  'jogállás', 'tulajdoni hányad', 'jogviszony'])

# to track which lines to write in
counter = 0

# Person ID - to track individuals
pid = int(0)


for i in range(len(reader.pages)):

    page = reader.pages[i]
    text = page.extract_text()
    t_lines = list(text.split('\n'))
    
    # to reach back and forward on the page if needed
    linecount = 0
    
    for line in t_lines:
        linecount += 1
        if '2. A nyilatkozatot adó neve:' in line:
            counter += 1
            pid += 1
            ing_sz = 0
            name = str(line.removeprefix('2. A nyilatkozatot adó neve: ').strip())
            
        if 'a) A település neve, ahol az ingatlan fekszik' in line:
            ing_sz += 1
            ing = t_lines[linecount]
            if ing_sz == 1:
                vny_2025.loc[counter, 'name'] = name
                vny_2025.loc[counter, 'PID'] = int(pid)
                vny_2025.loc[counter, 'ingatlan'] = ing
                vny_2025.loc[counter, 'ing_sz'] = ing_sz
                vny_2025.loc[counter, 'oldal'] = int(i+1)
                vny_2025.loc[counter, 'sor'] = int(linecount)
                
            elif ing_sz > 1:
                counter += 1
                vny_2025.loc[counter, 'name'] = name
                vny_2025.loc[counter, 'PID'] = int(pid)
                vny_2025.loc[counter, 'ingatlan'] = ing
                vny_2025.loc[counter, 'ing_sz'] = ing_sz
                vny_2025.loc[counter, 'oldal'] = int(i+1)
                vny_2025.loc[counter, 'sor'] = int(linecount)
                
                
        if 'b) Az ingatlan területnagysága:' in line:
            nm = t_lines[linecount]
            vny_2025.loc[counter, 'terület'] = nm

        if 'c) Művelési ága (vagy a művelés alól' in line:
            mua = t_lines[linecount]
            vny_2025.loc[counter, 'művelési ág'] = mua
            
        if 'd) Az épület fő rendeltetés szerinti jellege' in line:
            rend = t_lines[linecount+1]
            vny_2025.loc[counter, 'rendeltetés, alapterület'] = rend
            
        if 'e) Az ingatlan jogi jellege' in line:
            jj = t_lines[linecount+1]
            vny_2025.loc[counter, 'jogi jelleg'] = jj

        if 'f) A nyilatkozó jogállása (tulajdonos' in line:
            jog = t_lines[linecount]
            vny_2025.loc[counter, 'jogállás'] = jog

        if 'g) Közös tulajdon esetén a tulajdoni hányad' in line:
            tulh = str(line.removeprefix('g) Közös tulajdon esetén a tulajdoni hányad mértéke: '))
            vny_2025.loc[counter, 'tulajdoni hányad'] = tulh

        if 'h) A szerzés jogcíme, ideje (a jogviszony' in line:
            jogvisz = str(line.removeprefix('h) A szerzés jogcíme, ideje (a jogviszony kezdete): '))
            vny_2025.loc[counter, 'jogviszony'] = jogvisz

In [18]:
# letters in these columns will be converted to uppercase

to_uppercase = ['name', 'ingatlan', 'terület', 'művelési ág', 'rendeltetés, alapterület', 'jogi jelleg', 
                                  'jogállás', 'tulajdoni hányad', 'jogviszony']
for c in to_uppercase:
    vny_2025[c] = vny_2025[c].str.upper()

In [21]:
# save the file
vny_2025.to_csv('2025_ingatlanz.csv', index = False)